In [1]:
import torch
from efficientnet_pytorch import EfficientNet
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path
from cv2 import VideoCapture
import cv2
import os


In [2]:
working_dir = os.path.curdir
# you can find a pretrained model at model/b3.pth
MODEL_F = os.path.join(working_dir,'b0.pth')
# directory with the numpy optical flow images you want to use for inference
OF_NPY_DIR = os.path.join(working_dir, 'test_predictions')
#save flows to test_predictions folder as .npy files
try:
    os.mkdir(OF_NPY_DIR)
except:
    pass

In [3]:
#run optical flow

def calc_flow(img1,img2):
    return cv2.calcOpticalFlowFarneback(prev=img1, 
                                    next=img2, 
                                    flow=None,
                                    pyr_scale=0.5, 
                                    levels=3, 
                                    winsize=15,
                                    iterations=3, 
                                    poly_n=5, 
                                    poly_sigma=1.2, 
                                    flags=0)
def calc_all_flows(video):
    ret, prev_frame = video.read()
    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    flows = []
    count = 0
    while(video.isOpened()):
        count += 1
        if count % 30 == 0:
            print('Processed '+str(count) + ' frames (' +str(count//30) + ' seconds)')
        ret, next_frame = video.read()
        if ret == True:
            next_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
            flows.append(calc_flow(prev_frame, next_frame))
            prev_frame = next_frame
        else:
            break
    return flows

In [4]:
flows = calc_all_flows(VideoCapture(os.path.join(working_dir, 'mcgill_drive.mp4')))

In [5]:

for i in range(len(flows)):
    np.save(os.path.join(OF_NPY_DIR, 'flow'+str(i)+'.npy'), flows[i])

In [6]:
# check if cuda is available
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

## Load Model

In [7]:
V = 0     # what version of efficientnet did you use
IN_C = 2  # number of input channels
NUM_C = 1 # number of classes to predict

In [8]:
model = EfficientNet.from_pretrained(f'efficientnet-b{V}', in_channels=IN_C, num_classes=NUM_C)
state = torch.load(MODEL_F, map_location=device)
model.load_state_dict(state)
model.to(device)

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    2, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [9]:
def inference(of_f):
    of = np.load(of_f)
    print(of.shape)
    #change shape from (H,W,2) to (2,H,W)
    of = np.transpose(of, (2,0,1))
    #add batch dimension
    of = np.expand_dims(of, axis=0)
    i = torch.from_numpy(of).to(device)
    print('running')
    pred = model(i)
    print(pred)
    del i
    torch.mps.empty_cache()
    return pred

In [10]:
# loop over all files in directory and predict
# print(len(Path(OF_NPY_DIR).glob('*.npy')))
print()
for f in sorted(os.listdir(OF_NPY_DIR)):
    if f.endswith(".npy"):
        # print()
        y_hat = inference(os.path.join(OF_NPY_DIR,f)).item()
        print(f'{f}: {round(y_hat, 2)}')


(1440, 2562, 2)
running
tensor([[2.6789]], device='mps:0', grad_fn=<LinearBackward0>)
flow0.npy: 2.68
(1440, 2562, 2)
running
tensor([[2.8332]], device='mps:0', grad_fn=<LinearBackward0>)
flow1.npy: 2.83
(1440, 2562, 2)
running
tensor([[2.7007]], device='mps:0', grad_fn=<LinearBackward0>)
flow10.npy: 2.7
(1440, 2562, 2)
running
tensor([[1.9601]], device='mps:0', grad_fn=<LinearBackward0>)
flow100.npy: 1.96
(1440, 2562, 2)
running
tensor([[2.7141]], device='mps:0', grad_fn=<LinearBackward0>)
flow1000.npy: 2.71
(1440, 2562, 2)
running
tensor([[3.0140]], device='mps:0', grad_fn=<LinearBackward0>)
flow1001.npy: 3.01
(1440, 2562, 2)
running
tensor([[2.8811]], device='mps:0', grad_fn=<LinearBackward0>)
flow1002.npy: 2.88
(1440, 2562, 2)
running
tensor([[3.3094]], device='mps:0', grad_fn=<LinearBackward0>)
flow1003.npy: 3.31
(1440, 2562, 2)
running
tensor([[2.3181]], device='mps:0', grad_fn=<LinearBackward0>)
flow1004.npy: 2.32
(1440, 2562, 2)
running
tensor([[2.5254]], device='mps:0', grad_

KeyboardInterrupt: 